In [6]:
import sqlite3

In [7]:
START_YEAR = 2005
CURRENT_SEASON = 2024
COMPLETE_CURRENT_SEASON = False
MONTHS = [4,5,6,7,8,9] # March folded into April, October into September

In [8]:
db = sqlite3.connect("BaseballStats.db")
cursor = db.cursor()

In [9]:
from tqdm import tqdm

In [10]:
db.rollback()
cursor = db.cursor()

playerData = cursor.execute("SELECT * FROM Player_Hitter_MonthStats ORDER BY mlbId DESC, Year DESC, Month DESC, LevelId DESC").fetchall()
cursor.execute("BEGIN TRANSACTION")
cursor.execute("DELETE FROM Player_Hitter_MonthlyRatios")

for mlbId, year, month, level, ab, h, double, triple, hr, k, bb, sb, cs, hbp, parkFunFactor, ParkHRFacttor, GamesC, Games1B, Games2B, Games3B, GamesSS, GamesLF, GamesCF, GamesRF, GamesDH in tqdm(playerData):
    try:
        _, _, _, levelAvg, levelOBP, levelSLG, levelISO, levelWOBA, levelHRPerc, levelBBPerc, levelKPerc, levelSBRate, levelSBPerc = cursor.execute("SELECT * FROM Level_HitterStats WHERE LevelId=? AND Year=? AND Month=?", (level, year, month)).fetchone()
    except: # No Data
        continue
    
    pa = ab + bb + hbp
    singles = h - double - triple - hr
    if ab == 0:
        iso = levelISO
        avg = levelAvg
    else:
        iso = (double + 2 * triple + 3 * hr) / ab
        avg = h / ab
    if pa == 0:
        obp = levelOBP
        hrPerc = levelHRPerc
        bbPerc = levelBBPerc
        kPerc = levelKPerc
        sbRate = levelSBRate
        wOBA = levelWOBA
    else:
        # https://library.fangraphs.com/offense/woba/
        wOBA = (0.69 * bb + 0.72 * hbp + 0.89 * singles + 1.27 * double + 1.62 * triple + 2.10 * hr) / (pa)
        obp = (h + bb + hbp) / pa
        hrPerc = hr / pa
        bbPerc = bb / pa
        kPerc = k / pa
        sbRate = sb / pa
    
    if (sb + cs) != 0:
        sbPerc = sb / (sb + cs)
    else:
        sbPerc = levelSBPerc
    
    
    avgRatio = avg / levelAvg
    obpRatio = obp / levelOBP
    isoRatio = iso / levelISO
    wobaRatio = wOBA / levelWOBA
    sbRateRatio = sbRate / levelSBRate
    sbPercRatio = sbPerc / levelSBPerc
    hrPercRatio = hrPerc / levelHRPerc
    bbPercRatio = bbPerc / levelBBPerc
    kPercRatio = kPerc / levelKPerc
    totalGames = GamesC + Games1B + Games2B + Games3B + GamesSS + GamesLF + GamesCF + GamesRF + GamesDH
    PercC = GamesC / totalGames
    Perc1B = Games1B / totalGames
    Perc2B = Games2B / totalGames
    Perc3B = Games3B / totalGames
    PercSS = GamesSS / totalGames
    PercLF = GamesLF / totalGames
    PercCF = GamesCF / totalGames
    PercRF = GamesRF / totalGames
    PercDH = GamesDH / totalGames
    
    cursor.execute("INSERT INTO Player_Hitter_MonthlyRatios VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", (mlbId, year, month, level, avgRatio, obpRatio, isoRatio, wobaRatio, sbRateRatio, sbPercRatio, hrPercRatio, bbPercRatio, kPercRatio, PercC, Perc1B, Perc2B, Perc3B, PercSS, PercLF, PercCF, PercRF, PercDH))
    
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 430226/430226 [00:03<00:00, 129460.09it/s]
